In [28]:
library("networkD3")
library("dplyr")
library("tidyverse")
library("webshot")
library("htmlwidgets")
library("htmltools")

In [29]:
file = "/groups/dog/stage/victor/bamslam/csv/all_samples.csv"
outpath = "/groups/dog/stage/victor/bamslam/plots/sankey/"

In [30]:
df = read.csv(file)
df

sample,nbr_align,nbr_reads,nbr_unique_tr,med_cov_unique_tr,med_len_unique_tr,med_read_length,p_pcg_ref,p_pcg_new,p_lnc_ref,⋯,sec_align_mean_new_lncRNA,sec_align_mean_full,qscore_med_ref_protein_coding,qscore_med_ref_lncRNA,qscore_med_new_protein_coding,qscore_med_new_lncRNA,qscore_med_full,group,subgroup,nbr_unmapped_reads
<chr>,<int>,<int>,<int>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<int>
PC3_3_KM_PCa_R_R1_new_tr,29574380,6761251,113739,0.2815965,1903.0,864,0.8113494,0.02954631,0.01701386,⋯,5.109117,4.019848,11.296529,11.347226,11.259763,10.975024,11.284363,PC3,resistant,1661514
ADCA72_3_DF_lung_R_R1_new_tr,26095555,5714825,103164,0.2803313,1875.0,858,0.8060653,0.03247536,0.01373935,⋯,5.229995,4.204046,11.100775,11.181070,11.077029,10.830119,11.087682,ADCA72,resistant,1342513
PC3_3_KM_PCa_S_R1_new_tr,29977456,6558208,114490,0.2842037,1911.0,963,0.8074689,0.03118459,0.01571344,⋯,5.140827,4.136980,10.697591,10.810029,10.735311,10.512521,10.682230,PC3,sensitive,1703132
U251_S1_R1_new_tr,25546749,5661629,103036,0.2960467,1972.0,958,0.8139421,0.02862957,0.01426657,⋯,5.213547,4.158283,10.640616,10.617063,10.566022,10.296553,10.623827,U251,sensitive,1220627
501Mel_2_OSS_CM_R_R1_new_tr,27860908,6141176,116114,0.3093686,1876.0,956,0.8075685,0.03126144,0.01941159,⋯,5.065414,4.136631,10.546288,10.691764,10.455185,10.304742,10.544251,501Mel,resistant,1164191
501Mel_2_OSS_CM_S_R1_new_tr,39473560,8771842,123505,0.2699074,1798.0,938,0.7835732,0.03419544,0.01824258,⋯,5.077328,4.140598,9.892499,10.143993,9.930200,9.959069,9.894308,501Mel,sensitive,2549987
501Mel_1_OSS_CM_S_R1_new_tr,34576088,7807979,122375,0.3164557,1797.0,967,0.8075385,0.03080247,0.01815054,⋯,5.042943,4.070297,10.767713,10.919428,10.708973,10.550914,10.772636,501Mel,sensitive,1811677
ADCA72_1_DF_lung_R_R1_new_tr,24509515,5364787,97936,0.2997591,1891.0,930,0.8155535,0.03149109,0.01267842,⋯,5.227603,4.197484,11.153142,11.229576,11.121101,10.864750,11.140285,ADCA72,resistant,1114601
U251_TMZ_R2_R1_new_tr,38306380,8360232,118250,0.2862585,1847.0,890,0.8161265,0.02910099,0.01400356,⋯,5.218035,4.184632,12.463943,12.303103,12.367711,11.953676,12.457558,U251,resistant,1115129


In [31]:
df2 = df %>% dplyr::group_by(group, subgroup) %>%
dplyr::summarise(
    prop_total = 1,
    mean_prop_mapped_reads = mean(nbr_reads)/(mean(nbr_reads)+mean(nbr_unmapped_reads)),
    mean_prop_unmapped_reads = mean(nbr_unmapped_reads)/(mean(nbr_reads)+mean(nbr_unmapped_reads)),
    mean_prop_pcg_ref = mean(p_pcg_ref) * mean_prop_mapped_reads,
    mean_prop_pcg_new = mean(p_pcg_new) * mean_prop_mapped_reads,
    mean_prop_lnc_ref = mean(p_lnc_ref) * mean_prop_mapped_reads,
    mean_prop_lnc_new = mean(p_lnc_new) * mean_prop_mapped_reads,
    mean_prop_oth_ref = mean(p_oth_ref) * mean_prop_mapped_reads,
    mean_prop_oth_new = mean(p_oth_new) * mean_prop_mapped_reads,
    mean_prop_ref = sum(mean_prop_pcg_ref,mean_prop_lnc_ref,mean_prop_oth_ref),
    mean_prop_new = sum(mean_prop_pcg_new,mean_prop_lnc_new,mean_prop_oth_new)
    )
    

df2

`summarise()` has grouped output by 'group'. You can override using the `.groups` argument.


group,subgroup,prop_total,mean_prop_mapped_reads,mean_prop_unmapped_reads,mean_prop_pcg_ref,mean_prop_pcg_new,mean_prop_lnc_ref,mean_prop_lnc_new,mean_prop_oth_ref,mean_prop_oth_new,mean_prop_ref,mean_prop_new
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
501Mel,resistant,1,0.8370910,0.1629090,0.6766113,0.02574051,0.01635831,0.02505849,0.08336001,0.009962347,0.7763296,0.06076135
501Mel,sensitive,1,0.8035537,0.1964463,0.6441384,0.02537114,0.01474647,0.02640443,0.08143669,0.011456523,0.7403216,0.06323209
ADCA72,resistant,1,0.8200719,0.1799281,0.6684955,0.02524711,0.01069155,0.02181096,0.08317423,0.010652574,0.7623613,0.05771064
ADCA72,sensitive,1,0.8328302,0.1671698,0.6707352,0.02589816,0.01098996,0.02401298,0.08964525,0.011548658,0.7713704,0.06145980
PC3,resistant,1,0.8123114,0.1876886,0.6560563,0.02324867,0.01452063,0.02666258,0.07887905,0.012944164,0.7494560,0.06285541
PC3,sensitive,1,0.8034572,0.1965428,0.6544809,0.02321912,0.01200783,0.02464967,0.07678263,0.012317057,0.7432714,0.06018585
U251,resistant,1,0.8821491,0.1178509,0.7235175,0.02536839,0.01230128,0.02663404,0.08669577,0.007632040,0.8225146,0.05963447
U251,sensitive,1,0.8337756,0.1662244,0.6690848,0.02551285,0.01224632,0.02582186,0.08791389,0.013195903,0.7692450,0.06453061


In [32]:
for (i in 1:nrow(df2)){

mean_prop_mapped_reads=df2[i,]$mean_prop_mapped_reads
mean_prop_unmapped_reads=df2[i,]$mean_prop_unmapped_reads
mean_prop_ref=df2[i,]$mean_prop_ref
mean_prop_new= df2[i,]$mean_prop_new
mean_prop_pcg_ref=df2[i,]$mean_prop_pcg_ref
mean_prop_lnc_ref= df2[i,]$mean_prop_lnc_ref
mean_prop_oth_ref= df2[i,]$mean_prop_oth_ref
mean_prop_pcg_new= df2[i,]$mean_prop_pcg_new
mean_prop_lnc_new= df2[i,]$mean_prop_lnc_new
mean_prop_oth_new=df2[i,]$mean_prop_oth_new

if (df2[i,]$group == "501Mel"){
  cancer = "melanoma"
}
if (df2[i,]$group == "ADCA72"){
  cancer = "lung"
}
if (df2[i,]$group == "PC3"){
  cancer = "prostate"
}
if (df2[i,]$group == "U251"){
  cancer = "glioblastoma"
}


if (df2[i,]$subgroup == "sensitive"){
  type = "sensitive"
}
if (df2[i,]$subgroup == "resistant"){
  type = "resistant"
}

links <- data.frame(
  source=c("Total","Total", "Mapped", "Mapped", "Reference", "Reference", "Reference", "New", "New", "New"), 
  target=c("Mapped","Unmapped", "Reference", "New", "PCG Reference", "LNC Reference", "Other Reference", "PCG New", "LNC New", "Other New"), 
  value=c(mean_prop_mapped_reads, mean_prop_unmapped_reads, mean_prop_ref, mean_prop_new, mean_prop_pcg_ref, mean_prop_lnc_ref, mean_prop_oth_ref, mean_prop_pcg_new, mean_prop_lnc_new, mean_prop_oth_new)
  )

# From these flows we need to create a node data frame: it lists every entities involved in the flow
nodes <- data.frame(
  name=c(as.character(links$source), 
  as.character(links$target)) %>% unique()
)

# With networkD3, connection must be provided using id, not using real name like in the links dataframe.. So we need to reformat it.
links$IDsource <- match(links$source, nodes$name)-1 
links$IDtarget <- match(links$target, nodes$name)-1

# Add a 'group' column to the nodes and links data frame:
links$group <- as.factor(c("a1","a1","b1","b1","c1","d1","e1","c1","d1","e1"))
nodes$group <- as.factor(c("a","b","c","c","b","d","d","d","d","d","d"))
 
# Give a color for each group:
my_color <- 'd3.scaleOrdinal() .domain(["a1", "b1", "c1", "d1", "e1", "a", "b", "c", "d"]) .range(["#C2FFEC", "#F0FFB8", "#FFE9CC", "#FFE9CC", "#FFE9CC", "#1982c4", "#8ac926", "#ffc300", "#f18701"])'


# Make the Network
plot <- sankeyNetwork(Links = links, Nodes = nodes,
              Source = "IDsource", Target = "IDtarget",
              Value = "value", NodeID = "name",
              colourScale=my_color, NodeGroup="group", LinkGroup="group",
              sinksRight = FALSE,
              fontSize = 20,
              width=1000,height=600)

javascript_string <- 
  'function(el, x){
    d3.select(el).selectAll(".node text")
      .text(d => d.name + " (" + d.value.toFixed(2) + ")");
  }'


plot <- htmlwidgets::onRender(plot, jsCode = javascript_string)
plot <- htmlwidgets::prependContent(plot, htmltools::tags$h1(paste0("Repartition of reads in ",cancer, " ", type, " samples")))


outfile_html = paste0(outpath,"sankey_plot_",cancer,"_",type,".html")
outfile_png = paste0(outpath,"sankey_plot_",cancer,"_",type,".png")

# Save it as an html
saveWidget(plot, outfile_html)

# Convert it as png
webshot(outfile_html,outfile_png)

}